# **Модель для распознавания движений каратэномичи по углам конечностей (все файлы грузятся сслева)**

In [ ]:
#====================================================================
# @title 1.1 ЗАГРУЗКА МОДУЛЕЙ
#====================================================================
%%capture
!pip install mediapipe

In [ ]:
#====================================================================
# @title 1.2 ПОДКЛЮЧЕНИЕ БИБЛИОТЕК И МОДУЛЕЙ
#====================================================================
import os
import csv
import cv2
import time
import math
import gdown
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from google.colab.patches import cv2_imshow

In [ ]:
# @title Модули библиотеки sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score # Accuracy metrics
import pickle

In [ ]:
# @title Фреймворк для создания скелета
import mediapipe as mp # Import mediapipe

In [ ]:
# import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
mp_pose = mp.solutions.pose

In [ ]:
# Путь к корневому каталогу проекта
PATH = '/content'

In [ ]:
#====================================================================
# @title 1.3 ЗАГРУЗКА ДАННЫХ
#====================================================================

# Для обучения модели (Файл скачается у вас слева в /content/)
url1 = 'https://drive.google.com/uc?export=download&id=1nB74xWtuZ9WjV0MfTtlN7hod4bedJk3L'
gdown.download(url1, None)

Downloading...
From: https://drive.google.com/uc?export=download&id=1nB74xWtuZ9WjV0MfTtlN7hod4bedJk3L
To: /content/All_DataSet_Angles_v1_beta.csv
100%|██████████| 16.5M/16.5M [00:00<00:00, 57.5MB/s]


'All_DataSet_Angles_v1_beta.csv'

In [ ]:
#====================================================================
# @title 1.4 ПОДГОТОВКА ДАННЫХ
#====================================================================

# Загрузка CSV-файла в DataFrame и предпросмотр структуры данных
df_all = pd.read_csv(f'{PATH}/All_DataSet_Angles_v1_beta.csv')
df_all.head(len(df_all))

,class,view,belt,karateka,num,frame,ang_0_(28),ang_1_(27),ang_2_(26),ang_3_(25),ang_4_(24),ang_5_(23),ang_6_(24),ang_7_(23),ang_8_(12),ang_9_(11),ang_10_(14),ang_11_(13),ang_12_(12),ang_13_(11)
0,10,f,bb,1,1,0,68.027671,93.411735,153.343915,152.964913,121.917229,106.722422,149.741628,154.902789,23.456765,38.694997,113.453694,111.557852,27.921734,126.754987
1,10,f,bb,1,1,1,63.250852,99.588118,155.002795,146.597514,123.080573,113.343506,136.771270,156.516613,17.125390,34.871598,117.738190,112.961964,24.948903,127.745394
2,10,f,bb,1,1,2,64.616625,100.119555,153.702429,147.759633,121.643125,111.293121,140.296791,157.491690,17.520577,36.608854,115.429577,112.246262,24.189559,129.027131
3,10,f,bb,1,1,3,65.172063,101.226334,144.449909,146.904127,113.080469,113.601674,150.784772,156.911747,22.288098,40.132454,122.336058,121.672376,25.272209,129.701381
4,10,f,bb,1,1,4,65.635908,100.639907,135.489108,146.747837,105.129584,112.345056,159.315761,157.308864,23.057417,42.282019,120.487785,123.799862,25.542587,129.757664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60651,9,p,wb,1,10,39,79.444465,76.721456,120.800139,86.201738,115.676522,64.185314,130.464745,132.289451,53.822062,46.250372,61.954948,122.196390,71.829153,23.790627
60652,9,p,wb,1,10,40,81.660543,76.134504,123.303406,85.918423,115.836414,62.953187,129.210144,129.177035,59.455149,48.577335,62.248995,121.323524,78.813365,23.421537
60653,9,p,wb,1,10,41,80.154800,80.046721,119.289598,90.531210,117.752381,67.578400,125.357970,128.688307,56.153704,44.945343,58.109062,109.347921,76.290575,23.676837
60654,9,p,wb,1,10,42,84.175274,79.059612,124.111483,87.811061,119.128272,63.762821,124.119738,127.146860,54.335120,38.683973,54.860830,98.746206,70.497853,24.719237


In [ ]:
# Создадим копию DataFrame, чтобы не вносить изменения в оригинальный DataFrame.
df_copy = df_all.copy()

In [ ]:
# Список колонок с данными для обучения
columns = ['ang_0_(28)', 'ang_1_(27)', 'ang_2_(26)', 'ang_3_(25)', 'ang_4_(24)', 'ang_5_(23)', 'ang_6_(24)', 'ang_7_(23)', 'ang_8_(12)', 'ang_9_(11)', 'ang_10_(14)', 'ang_11_(13)', 'ang_12_(12)', 'ang_13_(11)']

In [ ]:
# Разделяем Датасет на данные и label.
x_data = df_copy[columns] # features
y_data = df_copy['class'] # target value

In [ ]:
# Разделение Датасета на выборки
x_train, x_test, y_train, y_test = train_test_split(x_data,                  # датасет с высотами и временем
                                                    y_data,                  # датасет с метками
                                                    test_size = 0.2,         # процент тестовых значений
                                                    # shuffle=True,            # перемешивание
                                                    random_state=42)         # параметр воспроизводимости. данные будут перемешаны в том же порядке. Чаще всего используют 0 или 42.

In [ ]:
#====================================================================
# @title 2.1 СОЗДАНИЕ МОДЕЛИ
#====================================================================

# Словарь-набор действий для создания 4 вариантов моделей для сравнения по точности
# Каждая строка словаря стандартизация данных и передача в модель
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()), # Логистическая регрессия
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()), # Классификатор Ridge
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()), # Классификатор случайный лес
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()), # Градиентный Бустинг Классификаторов
    'kn': make_pipeline(StandardScaler(), KNeighborsClassifier(3)) # Метод k ближайших соседей
}

In [ ]:
#====================================================================
# @title 2.2 ОБУЧЕНИЕ МОДЕЛИ
#====================================================================

# Создание словаря с уже обученными 4 моделями
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

In [ ]:
#====================================================================
# @title 2.3 РЕЗУЛЬТАТ И СТАТИСТИКА ОБУЧЕНИЯ МОДЕЛИ
#====================================================================

# Получение оценки точности моделей
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.3363831190240686
rc 0.30374216946917243
rf 0.7776953511374877
gb 0.5526706231454006
kn 0.7469502143092648


In [ ]:
#====================================================================
# @title 2.2 ФУНКЦИЯ ПРЕДОБРАБОТКИ ВИДЕО ДЛЯ АНАЛИЗА НС.
#====================================================================

In [ ]:
# @title Загрузка предварительных тестов

# параметры, /view?usp=sharing только при переходе по ссылке (но само видео должно быть расшарено!!!)!
# Файл скачается у вас слева в /content/
url1 = 'https://drive.google.com/uc?export=download&id=1MnL-zVxXMOP5blB7mj9Rg_WyiahKx2Uf'
gdown.download(url1, None)

url2 = 'https://drive.google.com/uc?export=download&id=1FUM88rH5fNyis4s_C1rdpeJCoM2JQa3o'
gdown.download(url2, None)

url3 = 'https://drive.google.com/uc?export=download&id=1lyNnmcjCYaJbKbvZ6ur--jvDTm2TAG9V'

gdown.download(url3, None)

Downloading...
From: https://drive.google.com/uc?export=download&id=1MnL-zVxXMOP5blB7mj9Rg_WyiahKx2Uf
To: /content/02_f_bb_01_03.mp4
100%|██████████| 5.06M/5.06M [00:00<00:00, 81.1MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1FUM88rH5fNyis4s_C1rdpeJCoM2JQa3o
To: /content/05_f_gb_02_02.mp4
100%|██████████| 5.52M/5.52M [00:00<00:00, 86.9MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1lyNnmcjCYaJbKbvZ6ur--jvDTm2TAG9V
To: /content/05_p_gb_02_02.mp4
100%|██████████| 6.07M/6.07M [00:00<00:00, 67.8MB/s]


'05_p_gb_02_02.mp4'

In [ ]:
# @title Загрузка (видео TG) тестов

# параметры, /view?usp=sharing только при переходе по ссылке (но само видео должно быть расшарено!!!)!
# Файл скачается у вас слева в /content/
url_TG0 = 'https://drive.google.com/uc?export=download&id=1IQ4hqKicxYtP6afcgIvqoA6S5Gn3EhaC'
gdown.download(url_TG0, 'test0_video.mp4')
url_TG1 = 'https://drive.google.com/uc?export=download&id=1N3tQJ9qwLVWqTJ94Ca2P8CdQ0_f5vX6M'
gdown.download(url_TG1, 'test1_video.mp4')
url_TG2 = 'https://drive.google.com/uc?export=download&id=1Ypcr-26ECjdQk92q9sX42Tln5K11qD2-'
gdown.download(url_TG2, 'test2_video.mp4')
url_TG3 = 'https://drive.google.com/uc?export=download&id=1d9bvaTt3I0RPfopZ0EHAy76Hy5HXqNme'
gdown.download(url_TG3, 'test3_video.mp4')
url_TG4 = 'https://drive.google.com/uc?export=download&id=1mdK8yT6IKSRSwbIKQjmhNGPKnnEQ6Wlr'
gdown.download(url_TG4, 'test4_video.mp4')
url_TG5 = 'https://drive.google.com/uc?export=download&id=1nUk1XkPnCdXRQ2l2wqe9biunOfzDIa7m'
gdown.download(url_TG5, 'test5_video.mp4')
url_TG6 = 'https://drive.google.com/uc?export=download&id=1bjgnP3XB88--rGr_AR5eLoNH13PEPLAv'
gdown.download(url_TG6, 'test6_video.mp4')
url_TG7 = 'https://drive.google.com/uc?export=download&id=1YhfESJrIemlGa62wD-9lw7J1qpECx25w'
gdown.download(url_TG7, 'test7_video.mp4')
url_TG8 = 'https://drive.google.com/uc?export=download&id=1dE9MmlbrvM20A6UqEa6w2kKH_JFN4Mue'
gdown.download(url_TG8, 'test8_video.mp4')
url_TG9 = 'https://drive.google.com/uc?export=download&id=10_RkL8TQM6WimvdVj62HxD4Aig8qXPpf'
gdown.download(url_TG9, 'test9_video.mp4')

Downloading...
From: https://drive.google.com/uc?export=download&id=1IQ4hqKicxYtP6afcgIvqoA6S5Gn3EhaC
To: /content/test0_video.mp4
100%|██████████| 4.20M/4.20M [00:00<00:00, 71.8MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1N3tQJ9qwLVWqTJ94Ca2P8CdQ0_f5vX6M
To: /content/test1_video.mp4
100%|██████████| 5.16M/5.16M [00:00<00:00, 83.1MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1Ypcr-26ECjdQk92q9sX42Tln5K11qD2-
To: /content/test2_video.mp4
100%|██████████| 3.14M/3.14M [00:00<00:00, 54.5MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1d9bvaTt3I0RPfopZ0EHAy76Hy5HXqNme
To: /content/test3_video.mp4
100%|██████████| 5.46M/5.46M [00:00<00:00, 78.9MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1mdK8yT6IKSRSwbIKQjmhNGPKnnEQ6Wlr
To: /content/test4_video.mp4
100%|██████████| 8.55M/8.55M [00:00<00:00, 41.2MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1nUk1XkPnCdX

'test9_video.mp4'

In [ ]:
#--------------------------------------------------------------------------
# @title Список точек для углов(угол относительно центральной)
#--------------------------------------------------------------------------

angles_Igor_Rybin = [[32, 28, 26], # Стопы
                     [31, 27, 25],
                     [28, 26, 24], # Колени
                     [27, 25, 23],
                     [26, 24, 23], # Пах
                     [25, 23, 24],
                     [26, 24, 12], # Изгиб тела
                     [25, 23, 11],
                     [24, 12, 14], # Плечи
                     [23, 11, 13],
                     [12, 14, 16], # Логти
                     [11, 13, 15],
                     [11, 12, 5], # Глаза
                     [12, 11, 2]
                    ]


In [ ]:
# @title Функция для вычисления углов

def calculate_triangle_angles(point1, point2, point3):
    # Вычисление длин сторон треугольника
    side_a = math.dist(point1, point2)
    side_b = math.dist(point2, point3)
    side_c = math.dist(point3, point1)

    # Вычисление углов треугольника с помощью формулы косинусов
    angle_A = math.degrees(math.acos((side_c**2 + side_a**2 - side_b**2) / (2 * side_c * side_a)))
    angle_B = math.degrees(math.acos((side_a**2 + side_b**2 - side_c**2) / (2 * side_a * side_b)))
    angle_C = math.degrees(math.acos((side_b**2 + side_c**2 - side_a**2) / (2 * side_b * side_c)))

    return angle_A, angle_B, angle_C, (side_a, side_b, side_c)

# Пример использования функции
point1 = [0, 0, 0]
point2 = [1, 0, 0]
point3 = [0, 1, 0]

triangle_angles = calculate_triangle_angles(point1, point2, point3)
print(triangle_angles)

(90.00000000000001, 44.999999999999986, 44.999999999999986, (1.0, 1.4142135623730951, 1.0))


In [ ]:
# @title Функция предобработки видео и формирование CSV.

def Preprocessing_Video(file_name_video_in, # Путь к входному видео
                        list_angles, # Список углов
                        file_name_video_out='video_out.mp4', # Путь к выходному видео):
                       ):
    # Счетчик кадров
    num_frame = 0

    # Характеристики модели, описывающей объект исследования
    landmarks = []

    # Определяем количество кадров в видеофайле
    video = VideoFileClip(file_name_video_in)
    count_frames = int(video.reader.nframes)

    # Список углов для описания тела человека
    angles = list_angles

    # Список колонок DataFrame
    columns = [f'ang_{i}_({col[1]})' for i, col in enumerate(angles)]

    df_ang = pd.DataFrame([], columns=columns)

    #--------------------------------------------------
    cap = cv2.VideoCapture(file_name_video_in)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    #--------------------------------------------------
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    # out = cv2.VideoWriter(file_name_video_out, fourcc, fps, (frame_width, frame_height))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(file_name_video_out, fourcc, fps, (frame_width, frame_height))

    with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Pose Detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                        )


            try:
                pose = results.pose_landmarks.landmark
                # pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
    #-------------------------------------------------
                angle_values = []

                for angle_list in angles:
                    i, j, k = angle_list
                    Point_First = [pose[i].x, pose[i].y, pose[i].z]
                    Point_Mid = [pose[j].x, pose[j].y, pose[j].z]
                    Point_End = [pose[k].x, pose[k].y, pose[k].z]

                    calc_angle = calculate_triangle_angles(Point_First, Point_Mid, Point_End)

                    angle_values.append(calc_angle[1])

                new_row = pd.Series(angle_values, index=df_ang.columns)
                # print('new_row', new_row)
                # df_ang = df_ang.append(new_row, ignore_index=True)
                df_new = pd.DataFrame([new_row])

                df_ang = pd.concat([df_ang, df_new], ignore_index=True)

            except Exception as e:
                print(f"Произошла ошибка: {e}")

            # Запись в видео
            out.write(image)

            num_frame +=1

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    # df_ang.to_csv(file_name_csv_angles, index=False)

    return df_ang

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
# @title Функция классификации удара

def Kicks_Classificator(predict):
    # Подсчет частоты появления каждого класса
    unique_classes, class_counts = np.unique(predict, return_counts=True)
    class_frequencies = dict(zip(unique_classes, class_counts))

    # Выбор класса с наибольшей частотой появления
    most_common_class = unique_classes[np.argmax(class_counts)]

    print("Частоты появления каждого класса:")
    print(class_frequencies)

    print("Класс с наибольшей частотой появления:")
    print(most_common_class)

    # Удары
    kicks = {1: '001-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)',
             2: '002-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)',
             3: '003-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)',
             4: '004-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)',
             5: '005-МАВАШИ ГЕРИ  (КРУГОВОЙ УДАР ЗАДНЕЙ НОГОЙ)',
             6: '006-МАЙ ГЕРИ КЕАГЕ (ПРЯМОЙ УДАР ЗАДНЕЙ НОГОЙ)',
             7: '007-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)',
             8: '008-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)',
             9: '009-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)',
             10: '010-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)'
            }
    kick = kicks[most_common_class]
    print('---------------------')
    print(f'Вы выполнили: {kick}')

In [ ]:
#====================================================================
# @title 2.2 РЕЗУЛЬТАТ ПРЕДСКАЗАНИЯ.
#====================================================================

**Это пока тестовый режим с отладочной информацией. Потом все `print()` закоментируем и будем писать ошибки в `LOG-файл`. На тестовых видео несколько ударов подряд, но они одинаковые, поэтому классификацияя работает! Надо сиамской бить на удары, чтобы наверняка! Тогда возможна классификация связок ударов!**

In [ ]:
%%time
ls_file = ['test0_video.mp4','test1_video.mp4', 'test2_video.mp4', 'test3_video.mp4', 'test4_video.mp4', 'test5_video.mp4'] + \
['test6_video.mp4','test7_video.mp4', 'test8_video.mp4', 'test9_video.mp4']
print('===================================')
for i, f in enumerate(ls_file):
    video_out = f'video_out_{i}.mp4'
    print('===================================')
    print(f'{f}')
    print(f'{video_out}')
    print('===================================')
    df_load = Preprocessing_Video(f, angles_Igor_Rybin, video_out)

    print('-----------------------------------')
    # Классификация "Случайный лес"
    class_predict_rf = fit_models['rf'].predict(df_load)
    # print(class_predict_rf)

    # РЕЗУЛЬТАТ РАСПОЗНАВАНИЯ УДАРА
    print("Случайный лес")
    Kicks_Classificator(class_predict_rf)
    print('-----------------------------------')
    # Классификация "Метод k ближайших соседей"
    class_predict_kn = fit_models['kn'].predict(df_load)
    # print(class_predict_kn)

    # РЕЗУЛЬТАТ РАСПОЗНАВАНИЯ УДАРА
    print("Метод k ближайших соседей")
    Kicks_Classificator(class_predict_kn)
    print('-----------------------------------')
print('===================================')

## **Работа с моделью**.

In [ ]:
# @title Сохраняем архитектуру и веса модели

with open('rf_kiks.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [ ]:
# @title Загружаем архитектуру и веса модели

with open('rf_kiks.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
# @title Файлы входные и обработаные
f  = '02_f_bb_01_03.mp4'
video_out = 'out_video.mp4'

In [ ]:
# @title Загружаем видео для предобработки (перевод в df: 1 кадр - 1 строка)

df_load = Preprocessing_Video(f, angles_Igor_Rybin, video_out)

In [ ]:
# @title Получаем значения классификации каждого кадра

predict_rf = model.predict(df_load)

In [ ]:
# @title Самый часто встречающийся класс и будет результатом классификации удара

Kicks_Classificator(predict_rf)

Частоты появления каждого класса:
{1: 1, 2: 42, 6: 1, 9: 1}
Класс с наибольшей частотой появления:
2
---------------------
Вы выполнили: 002-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)
